In [ ]:
import torch

In [ ]:
def reluToInplaceFalse(model):
  for name, child in model.named_children():
    if isinstance(child, nn.ReLU):
      setattr(child, 'inplace', False)
    else:
      reluToInplaceFalse(child)

from torchvision.transforms.transforms import RandomRotation

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = True, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone

    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
      
    # Disabling inplace ReLu becasuse GradCam doesn't work it enabled
    reluToInplaceFalse(backbone)
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.do_augmentation:
      self.augmentation = nn.Sequential(transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomPerspective(0.2),
                                        RandomRotation(20),
                                        transforms.RandomAutocontrast())

    if self.multi_backbone:
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)
    else:
      self.backbone =  nn.Sequential(*modules).to(device)


     

    self.fc1 = nn.Sequential(nn.Dropout(dropout_rate),
                              nn.Linear(self.out_channels, 128),
                              nn.ReLU(),
                              nn.Dropout(dropout_rate)) #TODO: Experiment with BN and Dropout

    # 512 features in, 3 features out
    self.fc = nn.Sequential(nn.Linear(512, 3))                  #TODO: L2 Regularization
     
  def forward(self, x, is_training = True):
    if self.do_augmentation and is_training:
      imgs = [self.augmentation(x[:,i]) for i in range(4)] #list of 4 images
    else:
      imgs = [x[:,i] for i in range(4)] #list of 4 images
      #imgs = [x[i] for i in range(4)]

    if self.multi_backbone:
        # feed each image into a backbone
      encodings = [self.fc1(torch.flatten(self.backbone1(imgs[0]),1)),
                   self.fc1(torch.flatten(self.backbone2(imgs[1]),1)),
                   self.fc1(torch.flatten(self.backbone3(imgs[2]),1)),
                   self.fc1(torch.flatten(self.backbone4(imgs[3]),1))]
    else:
      encodings = [self.fc1(self.backbone(img).squeeze()) for img in imgs]

    # modify to return an array with the largest encoding
    
    #raw_result = self.fc(torch.cat(encodings,1))
    #fixed_result = torch.clone(raw_result)
    #for idx, row in enumerate(raw_result):
    #    ones = torch.ones(3, dtype=torch.long)
    #    max_val = row.max()
    #    fixed_result[idx] = torch.where(row < max_val, 0, ones)
                  
    #return fixed_result.type(torch.int64)
    return self.fc(torch.cat(encodings,1))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = torch.load("drive/MyDrive/MNIST Feature Attribution/" + 'model_b_mar_7_00_05.pt', map_location="cpu")
model.eval()
model.zero_grad()

In [ ]:
print(torch.cuda.is_available())
# check that cuda is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [ ]:
model.to(device)

Classifier(
  (backbone1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): Re

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
x = torch.load("drive/MyDrive/MNIST Feature Attribution/" + "x_subset.tensor")
y = torch.load("drive/MyDrive/MNIST Feature Attribution/" + "y_subset.tensor")
dataset = TensorDataset(x,nn.functional.one_hot(y,3))

In [ ]:
dataloader = DataLoader(dataset,5,drop_last = True)

In [ ]:
!pip install -q captum

from captum.attr import *

     |████████████████████████████████| 1.4 MB 4.4 MB/s 


In [ ]:
ig = IntegratedGradients(model)

In [ ]:
all_attributions = []
# you do 5 at a time here
for images, labels in dataloader:
  # send images to the GPU
  images = images.to(device)
  # create new class-representative labels
  new_labels = []
  for t in labels.squeeze():
    if torch.equal(t, torch.tensor([1,0,0])):
      new_labels.append(0)
    elif torch.equal(t, torch.tensor([0,1,0])):
      new_labels.append(1)
    elif torch.equal(t, torch.tensor([0,0,1])):
      new_labels.append(2)
  new_labels = torch.tensor(new_labels,device=device)
  # get the attributions
  attributions = ig.attribute(images, target=new_labels, baselines = (images * 0), n_steps = 10, internal_batch_size = 10)
  all_attributions.append(attributions.cpu())
  del images, labels, attributions
  torch.cuda.empty_cache()

In [ ]:
# full
len(all_attributions)

10

In [ ]:
all_attributions[0].shape

torch.Size([5, 4, 3, 200, 1024])

In [ ]:
combined_attributions = torch.zeros(50, 4, 3, 200, 1024)

In [ ]:
t = [i*5 for i in range(11)]
pairs = zip(t[::2], t[1::2])

for idx, pair in enumerate(pairs):
  combined_attributions[pair[0]:pair[1]] = all_attributions[idx]

In [ ]:
combined_attributions.shape

torch.Size([50, 4, 3, 200, 1024])

In [ ]:
!pip install -q umap-learn[plot]

import umap

reducerFish = umap.UMAP(n_neighbors = 20,min_dist=0.5, verbose = True)
embeddingFish = reducerFish.fit_transform(torch.reshape(combined_attributions, (50, 2457600)))

     |████████████████████████████████| 86 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 7.6 MB/s 
     |████████████████████████████████| 15.8 MB 487 kB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 134 kB 46.0 MB/s 
     |████████████████████████████████| 837 kB 39.6 MB/s 
     |████████████████████████████████| 830 kB 49.4 MB/s 
     |████████████████████████████████| 822 kB 48.2 MB/s 
     |████████████████████████████████| 802 kB 48.1 MB/s 
     |████████████████████████████████| 802 kB 29.5 MB/s 
     |████████████████████████████████| 793 kB 34.4 MB/s 
     |████████████████████████████████| 793 kB 54.2 MB/s 
     |████████████████████████████████| 791 kB 39.3 MB/s 
     |████████████████████████████████| 786 kB 36.8 MB/s 
     |████████████████████████████████| 779 kB 45.6 MB/s 
     |████████████████████████████████| 778 kB 53.4 MB/s 
     |████████████████████████████████| 776 kB 14.5 MB/s 
     |█████████████

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Wed Mar  9 07:56:23 2022 Construct fuzzy simplicial set
Wed Mar  9 07:56:25 2022 Finding Nearest Neighbors
Wed Mar  9 07:56:29 2022 Finished Nearest Neighbor Search
Wed Mar  9 07:56:32 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Wed Mar  9 07:56:36 2022 Finished embedding


In [ ]:
combined_attributions.shape

torch.Size([50, 4, 3, 200, 1024])

In [ ]:
x.shape

torch.Size([50, 4, 3, 200, 1024])

In [ ]:
from io import BytesIO
import base64
from PIL import Image

from bokeh import plotting, palettes
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
import numpy as np
import pandas as pd

plotting.output_notebook()

In [ ]:
import matplotlib.colors as mplc

In [ ]:
def embeddableImage(data):
    # data should be [4, 3, 200, 1024] instance
    encoded_subimages = []
    for sub_image in data:
      # need to normalize each image to be between 0.0 to 1.0
      # can assume that everything is already a numpy array
      norm = mplc.Normalize(vmin = np.amin(sub_image), vmax = np.amax(sub_image))
      img_data = (norm(sub_image) * 255).astype(np.uint8).transpose(1, 2, 0)
      image = Image.fromarray(img_data, mode='RGB')
      buffer = BytesIO()
      image.save(buffer, format='png')
      encoded_subimages.append('data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode())
    return encoded_subimages

In [ ]:
import pandas as pd
import numpy as np

def umapPlot(embedding, x, y, yTrue=None, title=''):
    """ Plot the embedding of X and y with popovers using Bokeh """
    
    df = pd.DataFrame(embedding, columns=('x', 'y'))
    # for each image should be able to apply the embeddable image function
    # list of lists [rows x columns], x instances with 4 columns
    sub_images = np.array(list(map(embeddableImage, x)))
    for i in range(4):
      df['image'+str(i+1)] = sub_images[:,i]
    df['class'] = [str(d) for d in y]
    df['index'] = list(range(len(y)))
    if yTrue is not None:
        df['trueDigit'] = [str(d) for d in yTrue]

    datasource = ColumnDataSource(df)

    colorMapping = CategoricalColorMapper(factors=np.arange(10).astype(np.str), palette=palettes.Spectral10)

    plotFigure = plotting.figure(
        title=title,
        plot_width=600,
        plot_height=600,
        tools=('pan, wheel_zoom, reset')
    )

    if yTrue is None:
        tooltip = """
            <div>
                <div>
                    <img src='@image1' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image2' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image3' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image4' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Class:</span>
                    <span style='font-size: 18px'>@class</span>
                    <span style='font-size: 16px; color: #224499'>Index:</span>
                    <span style='font-size: 18px'>@index</span>
                </div>
            </div>
            """
    else:
        tooltip = """
            <div>
                <div>
                    <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Digit:</span>
                    <span style='font-size: 18px'>@digit (true: @trueDigit)</span>
                </div>
            </div>
            """
    plotFigure.add_tools(HoverTool(tooltips=tooltip))

    plotFigure.circle(
        'x', 'y',
        source=datasource,
        color=dict(field='class', transform=colorMapping),
        line_alpha=0.6, fill_alpha=0.6, size=8
    )
    plotting.show(plotFigure)
    
    return plotFigure

In [ ]:
fig = umapPlot(embeddingFish, combined_attributions.numpy(), y.squeeze().numpy(), title='UMAP projection of the Zebrafish dataset with Integrated Gradients Applied')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
combined_attributions.shape

torch.Size([50, 4, 3, 200, 1024])

In [ ]:
torch.save(combined_attributions,"drive/MyDrive/MNIST Feature Attribution/" +  "integrated_gradients_combined_attributions.tensor")